In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import cv2
from keras.preprocessing import image
categories = ['cats','dogs']
data = []
for category in categories:
    path = os.path.join('/kaggle/input/cats-and-dogs-image-classification/train',category)
    label = categories.index(category)
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label]) 

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
print(img_path)

/kaggle/input/cats-and-dogs-image-classification/train/dogs/dog_418.jpg


In [4]:
len(data)
import random
random.shuffle(data)
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)
# print(data)
import numpy as np
X = np.array(X)
y = np.array(y)
X.shape

(557, 224, 224, 3)

In [5]:
X = X/255

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify = y)
X_train.shape

(445, 224, 224, 3)

In [7]:
from keras.applications.vgg16 import VGG16
vgg = VGG16()
vgg.summary()

553467096/553467096 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [8]:
from keras import Sequential
model = Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [9]:
for layer in model.layers:
    layer.trainable = False

In [10]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [11]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test)) 

Epoch 1/5
14/14 [==============================] - 14s 351ms/step - loss: 0.7167 - accuracy: 0.5663 - val_loss: 0.6332 - val_accuracy: 0.6071
Epoch 2/5
14/14 [==============================] - 1s 96ms/step - loss: 0.5464 - accuracy: 0.7685 - val_loss: 0.5210 - val_accuracy: 0.8304
Epoch 3/5
14/14 [==============================] - 1s 96ms/step - loss: 0.4597 - accuracy: 0.8449 - val_loss: 0.4678 - val_accuracy: 0.8393
Epoch 4/5
14/14 [==============================] - 1s 96ms/step - loss: 0.4096 - accuracy: 0.8607 - val_loss: 0.4401 - val_accuracy: 0.7768
Epoch 5/5
14/14 [==============================] - 1s 96ms/step - loss: 0.3742 - accuracy: 0.8742 - val_loss: 0.4126 - val_accuracy: 0.8214


In [13]:
!pip install tensorflow
!pip install keras

In [14]:
import cv2 

In [25]:
Q = []
def detect_face_mask(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224))
    Q = np.array(img)
    Q = Q/255
    Q = Q.reshape(-1, 224, 224, 3)
    y_pred = model.predict(Q)
    if y_pred > 0.5:
        return "Dog"
    else:
        return "Cat"
aa = detect_face_mask('/kaggle/input/dogiee/dog.jpg')
print(aa)   

1/1 [==============================] - 0s 30ms/step
Dog
